In [ ]:
!apt-get update && apt-get install -y ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"
# !pip install "stable-baselines3[extra]"

In [ ]:
!pip install sb3-contrib

In [ ]:
!pip install wandb

In [ ]:
import numpy as np
print(f"{np.__version__=}")

In [ ]:
import stable_baselines3

print(f"{stable_baselines3.__version__=}")

## Imports

Stable-Baselines works on environments that follow the [gym interface](https://stable-baselines.readthedocs.io/en/master/guide/custom_env.html).
You can find a list of available environment [here](https://gym.openai.com/envs/#classic_control).

It is also recommended to check the [source code](https://github.com/openai/gym) to learn more about the observation and action space of each env, as gym does not have a proper documentation.
Not all algorithms can work with all action spaces, you can find more in this [recap table](https://stable-baselines.readthedocs.io/en/master/guide/algos.html)

In [ ]:
# !pip install moviepy

In [ ]:
# !pip install wandb

In [ ]:
# import moviepy
# print(f"{moviepy.__version__=}")

In [ ]:
# from IPython.core.display import HTML
# HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import gymnasium as gym
import numpy as np

print(f"{gym.__version__=}")

The first thing you need to import is the RL model, check the documentation to know what you can use on which problem

In [ ]:
from stable_baselines3.ppo import MlpPolicy

In [ ]:
# import gym
from gymnasium import logger, spaces
import math
import numpy as np
import random
import matplotlib.pyplot as plt
from gymnasium.spaces.discrete import Discrete
import io
import matplotlib
matplotlib.use('agg')  # turn off interactive backend
# matplotlib.use( 'plt' )
import numpy as np

from typing import Optional , Any
from matplotlib.patches import Wedge

# env_kwargs = {
#     render_mode : 'human',
#     max_episode : 100,
#     obs_detector_range : 0.5,

# }

class EversionRobot(gym.Env):
    metadata = {
        "render_modes": ["human", "rgb_array"],
        "render_fps": 15,
    }
    def __init__(
        self,
        obs_use : Optional[bool] = False,
        render_mode : Optional[str]='human',
        obs_detector_range: Optional[float]=0.5,
        max_step: Optional[int]=100,
        target: Optional[Any] = None
        ):
    #TODO: refactoring code
        super(EversionRobot, self).__init__()

        self.MAX_STEP = max_step
        self.target = target

        self.value_threshold = 5
        self.use_obstacle = obs_use

        self.obs_detector = [obs_detector_range for ind in range(7)]
        

#         self.action_space = spaces.Discrete(5)
        self.action_space = Discrete(5)
        

        self.steps_left = self.MAX_STEP

        self.low = [-2.5, 0.5]
        self.high = [2.5, 3.5]
        if self.target is None:
          self.x_target = [random.uniform(self.low[0],self.high[0]), random.uniform(self.low[1],self.high[1])]
        else:
          self.x_target = self.target
        # self.x_target = [-0.5, 0.5]

        # self.state = [0, 0, self.x_target[0], self.x_target[1], 0]
        self.state = None
        self.init_length = 0.1
        self.length = self.init_length
        self.bending = 0
        self.delta_length = 0.1
        self.delta_bending = 0.1
        self.segment_num = 1
        self.segment_num_max = 20
        self.length_max = 0.5
        self.length_array = []
        self.length_array.append(self.length)
        self.bending_array = []
        self.bending_array.append(self.bending)
        self.T_static = np.eye(3)
        self.x_tip = None
        self.y_tip = None

        self.safety_param = 2
        self.safety_penalty = 10

        self.render_mode = render_mode

        self.center = None
        self.radius_area = 0.3
        self.start_angle = None
        self.end_angle = None
        self.intersect_point = None
        self.intersection_length = self.obs_detector
        self.start_point = None
        self.distance = None
        if self.use_obstacle:
            high = np.array(self.obs_detector + [self.value_threshold, self.value_threshold, self.value_threshold, self.value_threshold, 2.0,10,10,10,20],
                        dtype=np.float32)
        else:
            high = np.array([self.value_threshold, self.value_threshold, self.value_threshold, self.value_threshold],
                        dtype=np.float32)
        self.observation_space = spaces.Box(-high, high, dtype=np.float32)
        # Obstacle's centre position and radius
        if self.use_obstacle:
            self.obs_center = []
            self.obs_center.append(np.array([0.0, 2.0]))
            self.obs_center.append(np.array([-1.1, 1.2]))
            # self.obs_center.append(np.array([0.1, 2.0]))
            self.obs_center.append(np.array([0.7, 1.0]))
            self.radius = []
            for i in range(0, len(self.obs_center)):
                self.radius.append(0.1)
        # if self.use_obstacle:
        #     self.obs_center = []
        #     for i in range(3):
        #       self.obs_center.append([random.uniform(-1.5,1), random.uniform(0.5,2)])
        #     self.radius = []
        #     for i in range(0, len(self.obs_center)):
        #         self.radius.append(0.1)
            while True:
              target_in_obs = False
              for i in range(len(self.obs_center)):
                if self.is_point_inside_circle(self.x_target[0],self.x_target[1],self.obs_center[i][0],self.obs_center[i][1],self.radius[i]):
                  target_in_obs = True
              if target_in_obs == True:
                self.x_target = [random.uniform(self.low[0],self.high[0]), random.uniform(self.low[1],self.high[1])]
              else:
                break
    def is_point_inside_circle(self,x, y, center_x, center_y, radius):
      distance = math.sqrt((x - center_x)**2 + (y - center_y)**2)
      return distance <= radius

    def constant_curvature(self, bending, length):
        if bending != 0:
            x_tip = (1-math.cos(bending))*length/bending
            y_tip = math.sin(bending)*length/bending
        else:
            x_tip = 0
            y_tip = length
        return x_tip, y_tip

    def static_segment(self, bending_array, length_array, segment_index):
        T_multi = np.eye(3)
        for i in range(0, segment_index):
            bending = self.bending_array[i]
            length = self.length_array[i]
            if bending != 0:
                x_tip = (1-math.cos(bending))*length/bending
                y_tip = math.sin(bending)*length/bending
            else:
                x_tip = 0
                y_tip = length
            T_single = np.array([[math.cos(bending), math.sin(bending), x_tip],
                                [-math.sin(bending), math.cos(bending), y_tip],
                                [0, 0, 1]])
            T_multi = T_multi@T_single

        return T_multi

    def pose_segment(self, segment_index):
        T_prior_segment = self.static_segment(self.bending_array, self.length_array, segment_index)
        indeks_maks = math.floor(self.length_array[segment_index]/self.delta_length)
        if indeks_maks!=0:
            bending_increment = self.bending_array[segment_index]/indeks_maks
        else:
            bending_increment = 0
        x_array = []
        y_array = []
        for i in range(0,indeks_maks):
            x_tip, y_tip = self.constant_curvature(bending_increment*i, self.delta_length*i)
            T_single = np.array([[math.cos(bending_increment*i), math.sin(bending_increment*i), x_tip],
                                [-math.sin(bending_increment*i), math.cos(bending_increment*i), y_tip],
                                [0, 0, 1]])
            transform_xy = T_prior_segment@T_single
            x_tip, y_tip = transform_xy[0,2], transform_xy[1,2]
            x_array.append(x_tip)
            y_array.append(y_tip)
        return x_array, y_array

    def check_collision(self):
        collision = False
        x_array, y_array = self.pose_segment(len(self.length_array)-1)
        for i in range(0, len(self.obs_center)):
            for j in range(0, len(x_array)):
                distance_vect = np.array([[x_array[j]-self.obs_center[i][0]],
                                            [y_array[j]-self.obs_center[i][1]]])
                distance_scalar = np.linalg.norm(distance_vect)
                if(distance_scalar<=self.radius[i]):
                    collision = True
                    break

            if(collision == True):
                break

        return collision

    def check_safety(self):
        danger = False
        x_array, y_array = self.pose_segment(len(self.length_array)-1)
        for i in range(0, len(self.obs_center)):
            for j in range(0, len(x_array)):
                distance_vect = np.array([[x_array[j]-self.obs_center[i][0]],
                                            [y_array[j]-self.obs_center[i][1]]])
                distance_scalar = np.linalg.norm(distance_vect)
                if(distance_scalar<=self.safety_param*self.radius[i]):
                    danger = True
                    break

            if(danger == True):
                break

        return danger
    def calculate_angle(self,point1, point2):
      x1, y1 = point1
      x2, y2 = point2
      # Calculate the angle in radians using arctan
      # angle_radians = math.atan(slope)
      angle_radians = math.atan2((y2 - y1), (x2 - x1))

      # Convert the angle from radians to degrees
      angle_degrees = math.degrees(angle_radians)

      return angle_degrees

    def calculate_endpoint(self,start_x, start_y, length, angle_degrees):
      # Convert angle from degrees to radians
      angle_radians = math.radians(angle_degrees)

      # Calculate endpoint coordinates
      end_x = start_x + length * math.cos(angle_radians)
      end_y = start_y + length * math.sin(angle_radians)

      return end_x, end_y

    def calculate_length_before_hit(self,start_x, start_y, angle_degrees, circle_center, circle_radius):
      line_length = 0.5
      # Parametric equations of the line
      # line_x = lambda t: start_x + t * math.cos(angle_radians)
      # line_y = lambda t: start_y + t * math.sin(angle_radians)
      # print(line_x)

      # Solve for t where the line intersects the circle
      # a = math.cos(angle_radians)**2 + math.sin(angle_radians)**2

      # Convert angle from degrees to radians
      angle_radians = math.radians(angle_degrees)

      a = 1
      b = 2 * (start_x * math.cos(angle_radians) - circle_center[0] * math.cos(angle_radians) + start_y * math.sin(angle_radians) - circle_center[1] * math.sin(angle_radians))
      c = start_x**2 - 2 * start_x * circle_center[0] + circle_center[0]**2 + start_y**2 - 2 * start_y * circle_center[1] + circle_center[1]**2 - circle_radius**2

      discriminant = b**2 - 4 * a * c

      if discriminant < 0:
          # No intersection
          return line_length
      else:
          # Calculate the values of t
          t1 = (-b + math.sqrt(discriminant)) / (2 * a)
          t2 = (-b - math.sqrt(discriminant)) / (2 * a)

          # Select the smallest positive root
          t = min(t1, t2)
          length_before_hit = t
          # Calculate the length before hitting the circle
          if(line_length < length_before_hit or length_before_hit < 0):
            return line_length
          else:
            return length_before_hit
    def calculate_circle_intersection(self,start_point, line_length, line_angle_degrees, circle_center, circle_radius,angle_robot):
        intersection_points = []
        intersection_length = []
        for angle in range(angle_robot,181+angle_robot,line_angle_degrees):
          # Calculate length before hitting the circle
          length_before_hit = self.calculate_length_before_hit(start_point[0], start_point[1], angle, circle_center, circle_radius)
          intersection_length.append(length_before_hit)
          # Calculate intersection point
          intersection_point = self.calculate_endpoint(start_point[0], start_point[1], length_before_hit, angle)
          intersection_points.append(intersection_point)
        return intersection_points, intersection_length


    def get_obstacle_detector_value(self):
        using_obs_detector = False
        # too expensive to calculate 2 segment point
        # x_array, y_array = self.pose_segment(len(self.length_array)-1)
        x_c,y_c = self.pose_segment(len(self.length_array)-1)[0][-1], self.pose_segment(len(self.length_array)-1)[1][-1]
        # x_c,y_c = self.x_tip, self.y_tip
        if(self.length>0.1):
          using_obs_detector = True

        if(using_obs_detector == True):
          # angle = self.calculate_angle((x_1,y_1),(x_2,y_2))
#           self.obs_det_bend += self.bending
          angle = 90-math.degrees(sum(self.bending_array))
          total_angle = 45
          start_angle = angle-(total_angle/2)
          end_angle = start_angle + total_angle
          center = (x_c,y_c)
          self.start_angle = start_angle
          self.end_angle = end_angle
          self.start_point = center
          angle_robot = angle - 90

          #experiment mode
          intersect_length = [0.5 for i in range(7)]
          intersect_point = [(5,5) for i in range(7)]
          for i in range(len(self.obs_center)):
            intersect_point_temp, intersect_length_temp= self.calculate_circle_intersection(self.start_point, 0.5, 30, self.obs_center[i], self.radius[i],int(angle_robot))
            for i in range(len(intersect_length)):
              if(intersect_length_temp[i] <= intersect_length[i]):
                intersect_length[i] = intersect_length_temp[i]
                intersect_point[i] = intersect_point_temp[i]

          self.intersection_length = intersect_length
          self.intersect_point = intersect_point
        return self.intersect_point, self.intersection_length

    def step(self, action):
        self.act = action
        save_state = self.state
        if action == 0:
            self.length = self.length+self.delta_length
        elif action == 1:
            self.length = self.length-self.delta_length
        elif action == 3:
            self.bending = self.bending+self.delta_bending
        elif action == 4:
            self.bending = self.bending-self.delta_bending
        if(self.bending>math.pi):
            self.bending = -math.pi
        elif(self.bending<-math.pi):
            self.bending = math.pi
        self.length = round(self.length,1)
        # Update length_array and bending_array
        if(self.length<=self.length_max and self.length>0):
            self.length_array[len(self.length_array)-1] = self.length
            self.bending_array[len(self.bending_array)-1] = self.bending
            # print(len(self.length_array))
        elif self.length>self.length_max:
            if(len(self.length_array)<=self.segment_num_max):
                self.length =  round(self.length - self.length_max,1)
                self.bending = 0
                self.length_array.append(self.length)
                self.bending_array.append(self.bending)
            else:
                self.length = self.length_max
        else:
            if(len(self.length_array)>1):
                self.length_array.pop()
                self.bending_array.pop()
                self.length = self.length_array[len(self.length_array)-1]
                self.bending = self.bending_array[len(self.bending_array)-1]
            else:
                self.length = 0

        self.T_static = self.static_segment(self.bending_array, self.length_array, len(self.bending_array))
        self.x_tip, self.y_tip = self.T_static[0,2], self.T_static[1,2]

        if self.use_obstacle:
            safety_flag = self.check_safety()
            if(safety_flag):
                safety_obs = 1
            else:
                safety_obs = 0
            self.flag_collision = self.check_collision()
            intersection_point , intersection_length = self.get_obstacle_detector_value()


            self.state = self.intersection_length + [self.x_tip, self.y_tip, self.x_target[0], self.x_target[1], safety_obs,self.length,self.bending,sum(self.length_array),sum(self.bending_array)]
        else:
            self.state = [self.x_tip, self.y_tip, self.x_target[0], self.x_target[1]]

        boundary= self.x_tip < -self.value_threshold or self.x_tip > self.value_threshold or self.y_tip < -self.value_threshold or self.y_tip > self.value_threshold

        error = np.array([self.x_tip,self.y_tip]) - np.array(self.x_target)
        self.distance = error

        done = bool(
            boundary
            or self.steps_left<0
            or self.length<0
        )
        if self.use_obstacle:
            done = done or self.flag_collision

        if not done:
            reward = - np.linalg.norm(error)**2
            if self.use_obstacle:
                reward = reward
        else:
            if self.length<0:
                reward = -100000
            elif self.use_obstacle:
                if self.flag_collision:
                    reward = -100000
                elif self.length<0:
                  reward = -100000
                else:
                    reward = 0
            else:
                reward = 0
        if not done:
            self.steps_left = self.steps_left-1
        self.cur_reward = reward
        self.cur_done = done
        return np.array(self.state, dtype=np.float32), reward, done,False, {}

    def reset(self,seed: Optional[int] = None):
        super().reset(seed=seed)
        if self.target is None:
          self.x_target = [random.uniform(self.low[0],self.high[0]), random.uniform(self.low[1],self.high[1])]
        else:
          self.x_target = self.target
        while True:
          target_in_obs = False
          for i in range(len(self.obs_center)):
            if self.is_point_inside_circle(self.x_target[0],self.x_target[1],self.obs_center[i][0],self.obs_center[i][1],self.radius[i]):
              target_in_obs = True
          if target_in_obs == True:
            self.x_target = [random.uniform(self.low[0],self.high[0]), random.uniform(self.low[1],self.high[1])]
          else:
            break
        # if self.use_obstacle:
        #     self.obs_center = []
        #     for i in range(3):
        #       self.obs_center.append([random.uniform(-1.5,1), random.uniform(0.5,2)])
        # self.x_target = [-0.5, 0.5]
        self.obs_detector = [0.5 for ind in range(7)]
        
        self.steps_left = self.MAX_STEP
        self.length = self.init_length
        self.bending = 0
        self.length_array = []
        self.length_array.append(self.length)
        self.bending_array = []
        self.bending_array.append(self.bending)
        self.T_static = np.eye(3)
        if self.use_obstacle:
            self.state = np.array(self.obs_detector + [self.value_threshold, self.value_threshold, self.value_threshold, self.value_threshold, 2.0,self.length,self.bending,sum(self.length_array),sum(self.bending_array)],
                        dtype=np.float32)
        else:
            self.state = [0, 0, self.x_target[0], self.x_target[1]]
        return np.array(self.state, dtype=np.float32) , {}

    def draw_segment(self, segment_index):
        if(segment_index%2==0):
            color_plot = 'red'
        else:
            color_plot = 'blue'
        x_array, y_array = self.pose_segment(segment_index)
        for i in range(0, len(x_array)):
            plt.scatter(x_array[i], y_array[i], color=color_plot)

    def draw_obs(self):
        for i in range(0, len(self.obs_center)):
            circle1 = plt.Circle((self.obs_center[i][0], self.obs_center[i][1]), self.radius[i], color='k')
            plt.gca().add_patch(circle1)

    def render(self, mode='human'):
        plt.axis('equal')
        # plt.xlim([-1.5, 1.5])
        # plt.ylim([-1.5, 1.5])
        for i in range(0, len(self.length_array)):
            self.draw_segment(i)

        if self.intersection_length is not None and self.start_point is not None and self.intersect_point is not None:
          for i in range(len(self.intersection_length)):
            color_line = 'b'
            if(self.intersection_length[i]<0.5):
              color_line = 'r'
            plt.plot([self.start_point[0], self.intersect_point[i][0]], [self.start_point[1], self.intersect_point[i][1]], marker='o', label='Line',color=color_line)
            plt.plot(self.intersect_point[i][0], self.intersect_point[i][1], marker='x', color=color_line, label='Intersection Point')

        plt.scatter(self.x_target[0], self.x_target[1], color='black')

        if self.use_obstacle:
            self.draw_obs()

        if self.render_mode == 'human':
            plt.draw()
            plt.pause(0.01)
        plt.xlabel('x (m)')
        plt.ylabel('y (m)')

        # Convert the plot to a NumPy array
        fig = plt.gcf()
        with io.BytesIO() as buff:
            fig.savefig(buff, format='raw')
            buff.seek(0)
            data = np.frombuffer(buff.getvalue(), dtype=np.uint8)
        w, h = fig.canvas.get_width_height()
        im = data.reshape((int(h), int(w), -1))

        plt.clf()
#         if self.use_obstacle:
#             print(f'State {self.state}, action: {self.act}, done: {self.cur_done}, bending: {self.bending_array}, length: {self.length_array}, reward: {self.cur_reward}, collision: {self.flag_collision}')
#         else:
#             print(f'State {self.state}, action: {self.act}, done: {self.cur_done}, bending: {self.bending_array}, length: {self.length_array}, reward: {self.cur_reward}')
        if self.render_mode == 'rgb_array':
            return im
        return 0

# Check env initialization

# WanDB initialization

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb_api")

wandb.login(key=wandb_api,relogin=True)

In [ ]:
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder
import wandb
from wandb.integration.sb3 import WandbCallback


config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 8000000,
    "env_name": "DQN-avoidance-test-discrete",
}
run = wandb.init(
    project="Eversion-robot_DQN",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    # monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
)


def make_env():
    env = EversionRobot(True,'rgb_array')
    env = Monitor(env)  # record stats such as returns
    return env


env = DummyVecEnv([make_env])
# env = VecVideoRecorder(
#     env,
#     f"videos/{run.id}",
#     record_video_trigger=lambda x: x % 200000 == 0,
#     video_length=100,
# )

# Model Training

In [ ]:
from typing import Callable
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

In [ ]:
from stable_baselines3 import DQN
from stable_baselines3 import PPO
# from sb3_contrib import QRDQN
# env = EversionRobot(True)
# model = DQN('MlpPolicy', env, verbose=1,device="cuda",tau=0.1,target_update_interval=100000,learning_rate=0.0001,exploration_final_eps=0.1,exploration_fraction=0.001,tensorboard_log=f"runs/{run.id}",seed=123456)
# model = DQN('MlpPolicy', env, verbose=1,device="cuda",tau=0.1,target_update_interval=100000,learning_rate=linear_schedule(0.0001)).learn(total_timesteps=100000,log_interval=10)
# model.save('DQN_test_avoidance')
model = DQN('MlpPolicy', env, verbose=0,device="cuda",tensorboard_log=f"runs/{run.id}",seed=123456)
# model = DQN('MlpPolicy', env, verbose=1,device="cuda",tau=0.1,target_update_interval=100000,learning_rate=linear_schedule(0.0001),tensorboard_log=f"runs/{run.id}")
# model = QRDQN('MlpPolicy', env, verbose=0,device="cuda",tensorboard_log=f"runs/{run.id}",seed=123456)

model.learn(
    total_timesteps=config["total_timesteps"],
    callback=WandbCallback(
        gradient_save_freq=100,
        model_save_path=f"models/{run.id}",
        verbose=0,
    )
)


In [ ]:
run.finish()

# Policy Evaluation

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
from stable_baselines3.common.monitor import Monitor
# Use a separate environement for evaluation
eval_env = Monitor(EversionRobot(True))

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100,deterministic=True)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
# # Set up fake display; otherwise rendering will fail
# import os
# os.system("Xvfb :1 -screen 0 1024x768x24 &")
# os.environ['DISPLAY'] = ':1'

# Visualize trained agent



In [ ]:
eval_env = DummyVecEnv([lambda: EversionRobot(True,'rgb_array')])
frames = []
video_length = 500
obs = eval_env.reset()
for le in range(video_length):
    # action, _ = model.predict(obs,deterministic=True)
    action, _ = model.predict(obs)
    obs, rewards, done, _ = eval_env.step(action)
    if(done):
      obs = eval_env.reset()
    print(rewards,action)
    frames.append(eval_env.render(mode='rgb_array'))
# Close the video recorder
eval_env.close()

In [ ]:
from matplotlib import pyplot as plt, animation
%matplotlib inline
from IPython import display

def create_anim(frames, dpi, fps):
    plt.figure(figsize=(frames[0].shape[1] / dpi, frames[0].shape[0] / dpi), dpi=dpi)
    patch = plt.imshow(frames[0])
    def setup():
        plt.axis('off')
    def animate(i):
        patch.set_data(frames[i])
    anim = animation.FuncAnimation(plt.gcf(), animate, init_func=setup, frames=len(frames), interval=fps)
    return anim

def display_anim(frames, dpi=72, fps=50):
    anim = create_anim(frames, dpi, fps)
    return anim.to_jshtml()

def save_anim(frames, filename, dpi=72, fps=50):
    anim = create_anim(frames, dpi, fps)
    anim.save(filename)

In [ ]:
display.HTML(display_anim(frames))

In [ ]:
save_anim(frames,'DQN_test.mp4')

# Basecode

In [ ]:
# # import gym
# from gymnasium import logger, spaces
# import math
# import numpy as np
# import random
# import matplotlib.pyplot as plt
# from gymnasium.spaces.discrete import Discrete
# import io
# import matplotlib
# matplotlib.use('agg')  # turn off interactive backend
# # matplotlib.use( 'plt' )
# import numpy as np

# from typing import Optional

# class EversionRobot(gym.Env):
#     metadata = {
#         "render_modes": ["human", "rgb_array"],
#         "render_fps": 15,
#     }
#     def __init__(self, obs_use : Optional[bool] = False,render_mode : Optional[str]='human'):
#         super(EversionRobot, self).__init__()
#         self.MAX_EPISODE = 100
#         self.x_threshold = 5
#         self.use_obstacle = obs_use

#         if self.use_obstacle:
#             high = np.array([self.x_threshold, self.x_threshold, self.x_threshold, self.x_threshold, 2.0],
#                         dtype=np.float32)
#         else:
#             high = np.array([self.x_threshold, self.x_threshold, self.x_threshold, self.x_threshold],
#                         dtype=np.float32)

# #         self.action_space = spaces.Discrete(5)
#         self.action_space = Discrete(5)
#         self.observation_space = spaces.Box(-high, high, dtype=np.float32)

#         self.steps_left = self.MAX_EPISODE
#         self.low = [-1.5, 0.5]
#         self.high = [1.5, 3.0]
#         self.x_target = [random.uniform(self.low[0],self.high[0]), random.uniform(self.low[1],self.high[1])]
#         # self.x_target = [-0.5, 0.5]
#         self.state = [0, 0, self.x_target[0], self.x_target[1], 0]
#         self.init_length = 0.1
#         self.length = self.init_length
#         self.bending = 0
#         self.delta_length = 0.1
#         self.delta_bending = 0.1
#         self.segment_num = 1
#         self.segment_num_max = 5
#         self.length_max = 1.5
#         self.length_array = []
#         self.length_array.append(self.length)
#         self.bending_array = []
#         self.bending_array.append(self.bending)
#         self.T_static = np.eye(3)
#         self.safety_param = 2
#         self.safety_penalty = 10
#         self.render_mode = render_mode

#         # Obstacle's centre position and radius
#         if self.use_obstacle:
#             self.obs_center = []
#             self.obs_center.append(np.array([0.0, 2.0]))
#             self.obs_center.append(np.array([-1.1, 1.2]))
#             self.obs_center.append(np.array([0.7, 1.0]))
#             self.radius = []
#             for i in range(0, len(self.obs_center)):
#                 self.radius.append(0.1)

#     def constant_curvature(self, bending, length):
#         if bending != 0:
#             x_tip = (1-math.cos(bending))*length/bending
#             y_tip = math.sin(bending)*length/bending
#         else:
#             x_tip = 0
#             y_tip = length
#         return x_tip, y_tip

#     def static_segment(self, bending_array, length_array, segment_index):
#         T_multi = np.eye(3)
#         for i in range(0, segment_index):
#             bending = self.bending_array[i]
#             length = self.length_array[i]
#             if bending != 0:
#                 x_tip = (1-math.cos(bending))*length/bending
#                 y_tip = math.sin(bending)*length/bending
#             else:
#                 x_tip = 0
#                 y_tip = length
#             T_single = np.array([[math.cos(bending), math.sin(bending), x_tip],
#                                 [-math.sin(bending), math.cos(bending), y_tip],
#                                 [0, 0, 1]])
#             T_multi = T_multi@T_single

#         return T_multi

#     def pose_segment(self, segment_index):
#         T_prior_segment = self.static_segment(self.bending_array, self.length_array, segment_index)
#         indeks_maks = math.floor(self.length_array[segment_index]/self.delta_length)
#         if indeks_maks!=0:
#             bending_increment = self.bending_array[segment_index]/indeks_maks
#         else:
#             bending_increment = 0
#         x_array = []
#         y_array = []
#         for i in range(0,indeks_maks):
#             x_tip, y_tip = self.constant_curvature(bending_increment*i, self.delta_length*i)
#             T_single = np.array([[math.cos(bending_increment*i), math.sin(bending_increment*i), x_tip],
#                                 [-math.sin(bending_increment*i), math.cos(bending_increment*i), y_tip],
#                                 [0, 0, 1]])
#             transform_xy = T_prior_segment@T_single
#             x_tip, y_tip = transform_xy[0,2], transform_xy[1,2]
#             x_array.append(x_tip)
#             y_array.append(y_tip)
#         return x_array, y_array

#     def check_collision(self):
#         collision = False
#         x_array, y_array = self.pose_segment(len(self.length_array)-1)
#         for i in range(0, len(self.obs_center)):
#             for j in range(0, len(x_array)):
#                 distance_vect = np.array([[x_array[j]-self.obs_center[i][0]],
#                                             [y_array[j]-self.obs_center[i][1]]])
#                 distance_scalar = np.linalg.norm(distance_vect)
#                 if(distance_scalar<=self.radius[i]):
#                     collision = True
#                     break

#             if(collision == True):
#                 break

#         return collision

#     def check_safety(self):
#         danger = False
#         x_array, y_array = self.pose_segment(len(self.length_array)-1)
#         for i in range(0, len(self.obs_center)):
#             for j in range(0, len(x_array)):
#                 distance_vect = np.array([[x_array[j]-self.obs_center[i][0]],
#                                             [y_array[j]-self.obs_center[i][1]]])
#                 distance_scalar = np.linalg.norm(distance_vect)
#                 if(distance_scalar<=self.safety_param*self.radius[i]):
#                     danger = True
#                     break

#             if(danger == True):
#                 break

#         return danger

#     def obstacle_vector(self, dist_to_goal):
#         limit_obs = 0.5
#         gain_obs = 3.0
#         x_array, y_array = self.pose_segment(len(self.length_array)-1)
#         min_distance = 100
#         min_index_i = 0
#         min_index_j = 0
#         distance_vect_min = np.array([[min_distance],[min_distance]])
#         for i in range(0, len(self.obs_center)):
#             for j in range(0, len(x_array)):
#                 distance_vect = np.array([[x_array[j]-self.obs_center[i][0]],
#                                             [y_array[j]-self.obs_center[i][1]]])
#                 distance_scalar = np.linalg.norm(distance_vect)
#                 if(distance_scalar<min_distance):
#                     min_distance = distance_scalar
#                     distance_vect_min = distance_vect
#                     min_index_i = i
#                     min_index_j = j

#         direction_obs = distance_vect_min/min_distance
#         if(np.linalg.norm(dist_to_goal)!=0):
#             direction_goal = dist_to_goal/np.linalg.norm(dist_to_goal)
#         else:
#             direction_goal = dist_to_goal
#         cos_angle = np.dot(direction_goal, direction_obs)
#         if((min_distance-self.radius[min_index_i])<=limit_obs and cos_angle>0):
#             distance_to_surface = min_distance - self.radius[min_index_i]
#             avoidance_term = (1.0/distance_to_surface - 1.0/limit_obs)*gain_obs
#         else:
#             avoidance_term = 0

#         return avoidance_term

#     def step(self, action):
#         self.act = action
#         save_state = self.state
#         if action == 0:
#             self.length = self.length+self.delta_length
#         elif action == 1:
#             self.length = self.length-self.delta_length
#         elif action == 3:
#             self.bending = self.bending+self.delta_bending
#         elif action == 4:
#             self.bending = self.bending-self.delta_bending
#         if(self.bending>math.pi):
#             self.bending = -math.pi
#         elif(self.bending<-math.pi):
#             self.bending = math.pi

#         # Update length_array and bending_array
#         if(self.length<=self.length_max and self.length>=0):
#             self.length_array[len(self.length_array)-1] = self.length
#             self.bending_array[len(self.bending_array)-1] = self.bending
#         elif self.length>self.length_max:
#             if(len(self.length_array)<=self.segment_num_max):
#                 self.length = 0
#                 self.bending = 0
#                 self.length_array.append(self.length)
#                 self.bending_array.append(self.bending)
#             else:
#                 self.length = self.length_max
#         else:
#             if(len(self.length_array)>1):
#                 self.length_array.pop()
#                 self.bending_array.pop()
#                 self.length = self.length_array[len(self.length_array)-1]
#                 self.bending = self.bending_array[len(self.bending_array)-1]
#             else:
#                 self.length = 0

#         self.T_static = self.static_segment(self.bending_array, self.length_array, len(self.bending_array))
#         x_tip, y_tip = self.T_static[0,2], self.T_static[1,2]

#         if self.use_obstacle:
#             safety_flag = self.check_safety()
#             if(safety_flag):
#                 safety_obs = 1
#             else:
#                 safety_obs = 0

#             self.state = [x_tip, y_tip, self.x_target[0], self.x_target[1], safety_obs]
#         else:
#             self.state = [x_tip, y_tip, self.x_target[0], self.x_target[1]]

#         boundary= x_tip < -self.x_threshold or x_tip > self.x_threshold or y_tip < -self.x_threshold or y_tip > self.x_threshold

#         error = np.array(self.state[0:2]) - np.array(self.x_target)

#         if self.use_obstacle:
#             self.flag_collision = self.check_collision()
#             reward_safety = -1*self.obstacle_vector(error)
#         done = bool(
#             boundary
#             or self.steps_left<0
#             or self.length<0
#         )
#         if self.use_obstacle:
#             done = done or self.flag_collision

#         if not done:
#             reward = - np.linalg.norm(error)**2
#             if self.use_obstacle:
#                 reward = reward + reward_safety
#         else:
#             if self.length<0:
#                 reward = -100000
#             elif self.use_obstacle:
#                 if self.flag_collision:
#                     reward = -100000
#                 else:
#                     reward = 0
#             else:
#                 reward = 0
#         if not done:
#             self.steps_left = self.steps_left-1
#         self.cur_reward = reward
#         self.cur_done = done
#         return np.array(self.state, dtype=np.float32), reward, done, False, {}

#     def reset(self,seed: Optional[int] = None):
#         super().reset(seed=seed)
#         self.x_target = [random.uniform(self.low[0],self.high[0]), random.uniform(self.low[1],self.high[1])]
#         # self.x_target = [-0.5, 0.5]
#         if self.use_obstacle:
#             self.state = [0, 0, self.x_target[0], self.x_target[1], 0]
#         else:
#             self.state = [0, 0, self.x_target[0], self.x_target[1]]
#         self.steps_left = self.MAX_EPISODE
#         self.length = self.init_length
#         self.bending = 0
#         self.length_array = []
#         self.length_array.append(self.length)
#         self.bending_array = []
#         self.bending_array.append(self.bending)
#         self.T_static = np.eye(3)
#         return np.array(self.state, dtype=np.float32) , {}

#     def draw_segment(self, segment_index):
#         if(segment_index%2==0):
#             color_plot = 'red'
#         else:
#             color_plot = 'blue'
#         x_array, y_array = self.pose_segment(segment_index)
#         for i in range(0, len(x_array)):
#             plt.scatter(x_array[i], y_array[i], color=color_plot)

#     def draw_obs(self):
#         for i in range(0, len(self.obs_center)):
#             circle1 = plt.Circle((self.obs_center[i][0], self.obs_center[i][1]), self.radius[i], color='k')
#             plt.gca().add_patch(circle1)

#     def render(self, mode='human'):
#         plt.axis('equal')
#         # plt.xlim([-1.5, 1.5])
#         # plt.ylim([-1.5, 1.5])
#         for i in range(0, len(self.length_array)):
#             self.draw_segment(i)
#         plt.scatter(self.x_target[0], self.x_target[1], color='black')
#         if self.use_obstacle:
#             self.draw_obs()
#         if self.render_mode == 'human':
#             plt.draw()
#             plt.pause(0.01)

#         # Convert the plot to a NumPy array
#         fig = plt.gcf()
#         with io.BytesIO() as buff:
#             fig.savefig(buff, format='raw')
#             buff.seek(0)
#             data = np.frombuffer(buff.getvalue(), dtype=np.uint8)
#         w, h = fig.canvas.get_width_height()
#         im = data.reshape((int(h), int(w), -1))

#         plt.clf()
# #         if self.use_obstacle:
# #             print(f'State {self.state}, action: {self.act}, done: {self.cur_done}, bending: {self.bending_array}, length: {self.length_array}, reward: {self.cur_reward}, collision: {self.flag_collision}')
# #         else:
# #             print(f'State {self.state}, action: {self.act}, done: {self.cur_done}, bending: {self.bending_array}, length: {self.length_array}, reward: {self.cur_reward}')
#         if self.render_mode == 'rgb_array':
#             return im
#         return 0